# Intro

This notebook introduces some of the functionality of the *stable-dracor-client* and the *dracor-sandbox* Docker container. It is assumed that the infrastructure has been started by using the docker compose file `compose.yml` (`docker compose up`) and run in the bundled jupyter lab instance on http://localhost:8888.

## DraCor in Docker and the *dracor-sandbox* Docker container

There are several possibilities to set up a local DraCor infrastructure. 

### DraCor Docker containers running on the local machine
One way would be to simply clone the repository of the DraCor API eXist-DB application (https://github.com/dracor-org/dracor-api) and use the provided docker compose file [`compose.yml`](https://github.com/dracor-org/dracor-api/blob/main/compose.yml) to start a local Docker-based DraCor system. If everything went well issuing the the command `docker ps` in a terminal will show multiple running Docker containers. 

(TODO: add screenshot here)

In this approach the environment in which the local infrastructure is running can vary. Depending on the setup of the local host machine (operation-system, version of Docker daemon installed, ...) there might be some issues, e.g. problems connection to `localhost` on some Windows installations. Although, the *stable-dracor-client* can be used with this setup as well, hence we will assume that the following *Docker in Docker* approach to setting up the local infrastructure will be used.

### Docker in Docker
Another way to setup a local DraCor infrastructure using Docker is to make use of a technique that is called *Docker in Docker*. This means that on the host machine there is running a container that has Docker installed. This Docker container is then used to start additional containers inside this controlled environment. This way the environment in which the DraCor services will be executed can be prepared in advanced to make sure that everything works as expected.

When issuing the command `docker compose up` in the root directory of the cloned stable-dracor repository only a single Docker container `dracor-sandbox` will be started. It not only has the Docker daemon installed in it but also features a (this) Jupyter Lab instance (running on http://localhost:8888) that can be used to prepare custom corpora or run Python scripts.

When starting the `dracor-sandbox` container there are no Docker containers running. To test this, it is possible to "enter" the container from the host machine with the command `docker exec -it dracor-sandbox /bin/bash`. In the interactive shell that is now executed inside the container the command `docker ps` will return an empty list. The `dracor-sandbox` container can be left with the command `exit`.

There are also two folders mounted to the container: `import` and `export`. They also show up in the *File Browser* in Jupyter Lab. These folder provide an easy way to get data into and out of the `dracor-sandbox` container. A usecase would be to add locally available XML files. When copying them into the `import` folder on the host machine, they become available to the container. The `export` folder functions the same way. Files that should become available on the host machine even when the Docker container is not running anymore can be stored in this folder. This could, for example, be the results of an analysis or a *manifest* that describes the composition of the custom corpora (see Section X).

Having only one Docker container with the DraCor system components inside also makes it possible to "freeze" not only the whole local Dracor system with the corpora set up (as we prototyped for the paper 
[Detecting Small Worlds in a Corpus of Thousands of Theater Plays](https://github.com/dracor-org/small-world-paper/tree/publication-version)) but in addition to that the environment that was used to assemble it (this Jupyter Lab instance with the Python notebooks).

In [4]:
# Activate Logging
import logging
#logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)

In [5]:
from stabledracor.client import StableDraCor

In [6]:
# check for GitHub Token (see FAQ Notebook)
import os
github_token = os.environ.get("GITHUB_TOKEN")
assert github_token is not None, "It is recommended to use a GitHub Personal Access Token. See FAQ Notebook for details."

In [7]:
dracor = StableDraCor(name="my-stable-dracor", github_access_token=github_token)

INFO:root:Initialized new StableDraCor instance: 'my-stable-dracor' (ID: ba657fe8-b3b2-4e68-a8f9-04c5c191b2b8).
INFO:root:Docker is available.
